# Charging stations for electric vehicles

## Libraries and settings

In [ ]:
# Libraries
import os
import re
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen

# Background color of graphics
# plt.style.use('dark_background')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

### Request charging station data

In [ ]:
# Url for server request
url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/oicp/ch.bfe.ladestellen-elektromobilitaet.json"
  
# Request data from server
response = urlopen(url)
  
# Load data
data_orig = json.loads(response.read())

# Save the data as pandas data frame
data = pd.DataFrame(data_orig["EVSEData"])
data

### Deriving data from the operator 'evpass'

In [ ]:
# Getting the index of evpass
idx = data[data["OperatorName"].isin(['evpass'])].index[0]
print(idx)

# Create data frame from column 'EVSEDataRecord' in the subset
df = pd.DataFrame(data.loc[idx, 'EVSEDataRecord'])
df.head()

### Take a closer look at the coordinates

In [ ]:
# Look at the geo-coordinates (latitude and longitude)
coordinates = df.GeoCoordinates
print(type(coordinates))
print(coordinates)

# Look at the type of a single item in the data frame
print(type(coordinates[0]))

### Extracting the latitude and longitude

In [ ]:
# Getting single values from as string
coord_str = "".join(coordinates[0].values())
coord_str 

# Splitting string by white space
re.findall(r'\S+', coord_str)

### Using a for loop to write the latitude and longitude to a separate data frame

In [ ]:
# Now loop though the coordinates object
out_list = []

for i in coordinates:
    # Getting single values from as string
    coord_str = "".join(i.values())
    # Splitting string by white space
    coords = re.findall(r'\S+', coord_str)
    out_list.append(coords)

# Create data frame from the list
df = pd.DataFrame(out_list, columns =['lat', 'lon'])
print(df.dtypes)

# Here we still have strings which need to be converted to integers

# First: exclude white space using pythons lambda function
df['lat'] = df['lat'].apply(lambda x: x.strip())
df['lon'] = df['lon'].apply(lambda x: x.strip())

# Second: cange to integer values
df['lat'] = df['lat'].astype(float)
df['lon'] =  df['lon'].astype(float)
df

### Plot the charging stations of the operator 'evepass'

In [ ]:
# Subset of gas stations by brand
locations = df[["lat", "lon"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]]).add_to(map)

# Plot map
map

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')